## 복수의 PCR Data File 전처리 
 + Raw Data Files := [pcr_raw-1.xls, pcr_raw-2.xls, pcr_raw-3.xls, pcr_raw-4.xls]
 + Signal 존재 여부 확률값 등 포함
 + pcr_pre.csv 파일에 누적


In [22]:
import matplotlib.pyplot as plt

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/gulim.ttc"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [23]:
import pandas as pd
import numpy as np
import keras


In [24]:
#--------------------------------------------------------- pcr_data.csv 파일 Loading
import xlrd

def loadRawData(filename) :
    workbook = xlrd.open_workbook(filename, ignore_workbook_corruption=True)  # --> 이렇게 해야 최신 Excel 파일 Read 가능
    df0 = pd.read_excel(workbook, header=None)#, skiprows=5)
    # df0 = pd.read_excel(filename, skiprows=7)#, encoding='utf8')  # --> 오류 발생

    machine_type = df0[df0[0] == 'Machine Type'][1].values[0]      #----------- PCR Machine Type 확인
    header_i = df0[df0[0] == 'Well'].index[0]                      #----------- 실제 Data만 Cutting
    df0.columns = ['Well Number', 'Well', 'Cycle', 'Target', 'Rn', 'dRn']
    df0 = df0[header_i+1:]
    return df0


## 전처리 --> Training Data 준비

### Signal 판정 기준표 Loading

In [25]:
guideline_file = "Signal 판정 Guideline.xlsx"

df_guide = pd.read_excel(guideline_file, skiprows=6)
df_guide

,Ct_Rn_Times,Ct_Rn,P_Ct_Rn,Th_Ct_Diff,Th_Ct,P_Th_Ct,Rn_Last_Times,Rn_Last,P_Rn_Last
0,0.05,0.01,0.00,-6,24,1.00,0.5,0.1,0.00
1,0.10,0.02,0.03,-5,25,0.96,1.0,0.2,0.03
2,0.20,0.04,0.10,-4,26,0.88,2.0,0.4,0.10
3,0.40,0.08,0.20,-3,27,0.80,2.5,0.5,0.20
4,0.60,0.12,0.28,-2,28,0.72,3.0,0.6,0.28
5,0.80,0.16,0.38,-1,29,0.62,3.5,0.7,0.38
6,1.00,0.20,0.50,0,30,0.50,5.0,1.0,0.50
7,1.30,0.26,0.62,1,31,0.40,6.0,1.2,0.62
8,1.60,0.32,0.72,2,32,0.32,7.0,1.4,0.72
9,1.90,0.38,0.80,3,33,0.25,8.0,1.6,0.80


In [26]:
base = {}
base['Ct_Rn_Times'] = df_guide['Ct_Rn_Times'].values
base['P_Ct_Rn'] = df_guide['P_Ct_Rn'].values
base

{'Ct_Rn_Times': array([0.05, 0.1 , 0.2 , 0.4 , 0.6 , 0.8 , 1.  , 1.3 , 1.6 , 1.9 , 2.2 ,
        2.5 , 2.8 ]),
 'P_Ct_Rn': array([0.  , 0.03, 0.1 , 0.2 , 0.28, 0.38, 0.5 , 0.62, 0.72, 0.8 , 0.88,
        0.96, 1.  ])}

### + CT_Rn 값으로 Signal 유무 판단

In [27]:
Th = 0.2
Ct_Cycle = 30     # 증폭 Cycle 30번에서 dRn 값이 Th 값을 넘는가?
Rn_Last_Base = 5  # Rn(40) 값의 기준은 Th 값의 5배

def P_Ct_Rn(data):
    base = {}
    base['Ct_Rn_Times'] = df_guide['Ct_Rn_Times'].values.tolist()
    base['P_Ct_Rn'] = df_guide['P_Ct_Rn'].values.tolist()
    
    Ct_Rn = data[Ct_Cycle - 1]
    P_Ct_Rn = 1.0
    
    for i in range(0, len(base['P_Ct_Rn'])):
        if Ct_Rn <= base['Ct_Rn_Times'][i] * Th :
            P_Ct_Rn = base['P_Ct_Rn'][i]
            break
            
    return P_Ct_Rn
        

In [28]:
# Th = 0.2
# Ct_Cycle = 30     # 증폭 Cycle 30번에서 dRn 값이 Th 값을 넘는가?
# Rn_Last_Base = 5  # Rn(40) 값의 기준은 Th 값의 5배

# def P_Ct_Rn(data):
#     base_table = [[0.2, 0], [0.4, 0.01], [0.6, 0.03], [0.7, 0.1], [0.8, 0.2], [0.9, 0.35], [1, 0.6], [1.3, 0.78], [1.5, 0.9], [1.7, 0.95], [1.9, 0.98], [2.1, 0.99], [2.5, 1]]
    
#     Ct_Rn = data[Ct_Cycle - 1]
#     P_Ct_Rn = 1.0
    
#     for i in range(0, len(base_table)):
#         if Ct_Rn <= base_table[i][0] * Th :
#             P_Ct_Rn = base_table[i][1]
# #             print (f"CT_Rn({Ct_Rn} --> Signal 확률 = {P_Ct_Rn})")
#             break
            
#     return P_Ct_Rn
        

### + Th 값 최초 초과 CT Cycle #로  Signal 유무 판단

In [29]:
def P_Th_Ct(data):
    base = {}
    base['Th_Ct_Diff'] = df_guide['Th_Ct_Diff'].values.tolist()
    base['P_Th_Ct'] = df_guide['P_Th_Ct'].values.tolist()
   
    Th_Ct = 6
    p_Th_Ct = 0.0
    for i in range(0, len(data)) :
        if data[i] >= Th :
            Th_Ct = i + 1
            diff_Th_Ct = Th_Ct - Ct_Cycle
            if diff_Th_Ct < -6 :
                diff_Th_Ct = -6
            if diff_Th_Ct > 6 :
                diff_Th_Ct = 6
                
            p_Th_Ct = base['P_Th_Ct'][ base['Th_Ct_Diff'].index(diff_Th_Ct) ]
            return p_Th_Ct
    
    return p_Th_Ct


In [30]:
# def P_Th_Ct(data):
#     base_table = { -6 : 1, -5 : 0.99, -4 : 0.98, -3 : 0.96, -2 : 0.85, -1 : 0.75, 0 : 0.6, 1 : 0.45, 2 : 0.3, 3 : 0.1, 4 : 0.03, 5 : 0.01, 6 : 0}
    
#     Th_Ct = 6
#     P_Th_Ct = 0.0
#     for i in range(0, len(data)) :
#         if data[i] >= Th :
#             Th_Ct = i + 1
#             diff_Th_Ct = Th_Ct - Ct_Cycle
#             if diff_Th_Ct < -6 :
#                 diff_Th_Ct = -6
#             if diff_Th_Ct > 6 :
#                 diff_Th_Ct = 6
                
#             P_Th_Ct = base_table[diff_Th_Ct]
# #             print (f"Th_Ct({Th_Ct} --> Signal 확률 = {P_Th_Ct})")
#             return P_Th_Ct
    
# #     print (f"Th_Ct(Th 도달 못함) --> Signal 확률 = {P_Th_Ct})")
#     return P_Th_Ct


### Last Cycle dRn 값으로  Signal 유무 판단

In [31]:
def P_Rn_Last(data):
    base = {}
    base['Rn_Last_Times'] = df_guide['Rn_Last_Times'].values.tolist()
    base['P_Rn_Last'] = df_guide['P_Rn_Last'].values.tolist()
    
    Rn_Last = data[len(data) - 1]
    P_Rn_Last = 1.0
    
    for i in range(0, len(base['P_Rn_Last'])):
        if Rn_Last <= base['Rn_Last_Times'][i] * Th :
            P_Rn_Last = base['P_Rn_Last'][i]
            break
            
    return P_Rn_Last


In [32]:
# def P_Rn_Last(data):
#     base_table = [[0.5, 0], [1, 0.01], [2, 0.03], [2.5, 0.1], [3, 0.2], [3.5, 0.35], [5, 0.6], [6, 0.78], [7, 0.9], [8, 0.96], [9, 0.98], [10, 0.99], [11, 1]]
    
#     Rn_Last = data[len(data) - 1]
#     P_Rn_Last = 1.0
    
#     for i in range(0, len(base_table)):
#         if Rn_Last <= base_table[i][0] * Th :
#             P_Rn_Last = base_table[i][1]
# #             print (f"Rn_Last({Rn_Last} --> Signal 확률 = {P_Rn_Last})")
#             break
            
#     return P_Rn_Last


### 세가지 조건에 따른 Signal 확률 조합하기

In [33]:

def P_FinalMax(_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last, _sel = 1) :
    #===================== Max ====================
    #-------------------------------------------  3 값 중 --> Max 값
    if _sel == 1 :
        _P = max([_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last])
    
    #-------------------------------------------  Ct_Rn & Rn_Last --> Max 값
    if _sel == 2 :
        _P = max([_P_Ct_Rn, _P_Rn_Last])
    
    #-------------------------------------------  Th_Ct & Rn_Last  --> Max 값
    if _sel == 3 :
        _P = max([_P_Th_Ct, _P_Rn_Last])
        
    return _P
    

    
def P_FinalMean(_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last, _sel = 1) :
    #===================== Max ====================
    #-------------------------------------------  전체  평균 값
    if _sel == 1 :
        _P = np.mean([_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last])

    #-------------------------------------------  Ct_Rn & Rn_Last  평균 값
    if _sel == 2 :
        _P = np.mean([_P_Ct_Rn, _P_Rn_Last])

    #-------------------------------------------  Th_Ct & Rn_Last  평균 값
    if _sel == 3 :
        _P = np.mean([_P_Th_Ct, _P_Rn_Last])
        
    return np.round(_P, 2)

        
def P_FinalOX(_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last, _sel = 1) :
    #===================== Digital : O or 1 ====================
    #-------------------------------------------  3 값 중 하나만 0.5 이상이면
    if _sel == 1 :
        _P = max([_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last])
        if _P >= 0.5 :
            return 1
        else :
            return 0
    
    #-------------------------------------------  Ct_Rn or Rn_Last  > 0.5
    if _sel == 2 :
        _P = max([_P_Ct_Rn, _P_Rn_Last])
        if _P >= 0.5 :
            return 1
        else :
            return 0
    
    #-------------------------------------------  Th_Ct or Rn_Last  > 0.5
    if _sel == 3 :
        _P = max([_P_Th_Ct, _P_Rn_Last])
        if _P >= 0.5 :
            return 1
        else :
            return 0

    #-------------------------------------------  기존 방식 Ct_Rn 값 만으로.....
    if _sel == 4 :
        if _P_Ct_Rn >= 0.5 :
            return 1
        else :
            return 0


### Meta Data (Signal 확률 생성)

In [34]:
def changeDataFrameForTrainingWithProbability(df0):
    _cols = [i for i in range(1,41)]
#     _cols = ["Set", "Sample", "Target", "P_Final_OX", "P_Final_Mean", "P_Final_Max", "P_Ct_Rn", "P_Th_Ct", "P_Rn_Last"] + _cols
    _cols = ["Target", "P_Final_OX", "P_Final_Mean", "P_Final_Max", "P_Ct_Rn", "P_Th_Ct", "P_Rn_Last"] + _cols
    df = pd.DataFrame(columns = _cols)

    for sample in range(1, 13) :
        for i in range(65, 73) :
            well =f"{chr(i)}{sample}" 
            for target in range(1, 5) :
                vals = df0[(df0['Well'] == well) & (df0['Target'] == f'Target {target}')]["dRn"].values
                _P_Ct_Rn   = P_Ct_Rn(vals)
                _P_Th_Ct   = P_Th_Ct(vals)
                _P_Rn_Last = P_Rn_Last(vals)
                _P_FinalMax = P_FinalMax(_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last, _sel=1)
                _P_FinalMean = P_FinalMean(_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last, _sel=1)
                _P_FinalOX = P_FinalOX(_P_Ct_Rn, _P_Th_Ct, _P_Rn_Last, _sel=4)
                
                _vals = [f"{chr(i)}{sample}-{target}", _P_FinalOX, _P_FinalMean, _P_FinalMax, _P_Ct_Rn, _P_Th_Ct, _P_Rn_Last] + vals.tolist()

                df.loc[len(df)] = _vals
    return df
# print(cols) # , end=" ")



### 복수의 Raw Data 파일들 전처리 --> pcr_pre.csv에 누적

In [35]:
raw_files = ["pcr_raw-1.xls", "pcr_raw-2.xls", "pcr_raw-3.xls", "pcr_raw-4.xls"]
filename_pre = "pcr_pre.csv"

is_first = True
def preProcess(filename_raw) :
    global is_first
    #--------------------------------------------------------- Load Raw Data File
    df0 = loadRawData(filename_raw)

    #--------------------------------------------------------- Transform & Preprocessing
    tdf = changeDataFrameForTrainingWithProbability(df0)

    #--------------------------------------------------------- 전처리 데이터 저장 to pcr_pre.csv
    if is_first == True :
        tdf.to_csv(filename_pre, encoding='euckr', index=False, mode='a')
        is_first = False
    else:
        tdf.to_csv(filename_pre, encoding='euckr', index=False, mode='a')  #****************************************
    
    return tdf


for raw_file in raw_files :
    print(f"{raw_file} 파일 Data 전처리 ...... ---> pcr_pre.csv")
    tdf = preProcess(raw_file)
#     break


pcr_raw-1.xls 파일 Data 전처리 ...... ---> pcr_pre.csv
pcr_raw-2.xls 파일 Data 전처리 ...... ---> pcr_pre.csv
pcr_raw-3.xls 파일 Data 전처리 ...... ---> pcr_pre.csv
pcr_raw-4.xls 파일 Data 전처리 ...... ---> pcr_pre.csv


In [21]:
df_pre = pd.read_csv(filename_pre, encoding='euckr')
df_pre

ParserError: Error tokenizing data. C error: Expected 46 fields in line 386, saw 47


### 통계적 판정결과 점검

In [19]:
n_columns = 6  # or 8
draw_from = 20
def drawSignalResult(df) :  # pcr_pre.csv
    count = 0
    for i, row in df.iterrows() :
        p_final_ox   = row['P_Final_OX']
        p_final_mean = row['P_Final_Mean']
        p_final_max  = row['P_Final_Max']
        p_ct_rn      = row['P_Ct_Rn']
        p_th_ct      = row['P_Th_Ct']
        p_rn_last    = row['P_Rn_Last']
        data         = row.values[6+draw_from:]
        legend       = f"{row['Target']}({i+1})"
        cycles       = [i for i in range(draw_from,  41)]
        
#         print(data)
        
        if count == 0 :
            plt.subplots(figsize=(15, 2))
        count += 1

        plt.subplot(1, n_columns, count)
        plt.plot(cycles, data, color="dodgerblue", alpha=1)
        plt.title(f"{p_final_ox},  {p_final_mean},  {p_final_max} / {p_ct_rn},  {p_th_ct},  {p_rn_last}")
        plt.legend([legend])
        plt.ylim(0, 7)
        plt.grid(True, color='gray', alpha=0.3, linestyle='--') # , axis='y')

        if count >= n_columns :
            plt.tight_layout()
            plt.show()
            count = 0

drawSignalResult(df_pre)


KeyError: 'Target'